In [1]:
import tdmclient.notebook
await tdmclient.notebook.start()
from tdmclient import ClientAsync, aw
client = ClientAsync()
client.process_waiting_messages()
node = await client.wait_for_node()
aw(node.run())
aw(node.stop())
aw(node.set_scratchpad("Hello, Studio!"))
aw(node.unlock())

import time
from aled import  *
from l42 import *


In [ ]:
initial_corners = init_corner()

@tdmclient.notebook.sync_var
def get_prox_value():
    return prox_horizontal

@tdmclient.notebook.sync_var
def get_ground_value():
    return prox_ground_reflected

@tdmclient.notebook.sync_var
def get_motors_speed():
    global motor_left_target, motor_right_target
    return motor_left_target, motor_right_target


@tdmclient.notebook.sync_var
def set_motor_speed(right_speed, left_speed):
    global motor_left_target, motor_right_target
    motor_left_target = left_speed
    motor_right_target = right_speed


while(True): 
    #copy the states of the robot   
    prox_horiz_values = get_prox_value()
    prox_ground_values = get_ground_value()
    set_motor_speed(0, 0)
    motors_speeds = get_motors_speed()
        
    #open camera and update of the KF + color detection    
    if(not open_cam()):
        break
    await client.sleep(0.2)




[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[4159, 3658, 2764, 2821, 0, 0, 0]
[4634, 3595, 4414, 3745, 0, 0, 0]
[4682, 3959, 4205, 3850, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]


In [ ]:
await tdmclient.notebook.stop()